# Testing homemade layers

This notebook contains tests for all the homemade layers. The tests takes some samples from the mni-
st dataset as a minibatch and compares the results from running this minibatch through the test CNN
to running the same minibatch through the homemade layers.

In [33]:
#importing relevant libraries

import numpy as np
import torch
import torch.nn as nn
import torchvision
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, TensorDataset, Subset
import torch.optim as optim
import torch.nn.functional as F
import os

from CNN_small_architecture import CNNSmall
from CNN_layers import conv_homemade
from CNN_layers import maxpool_homemade
from CNN_layers import batchnorm_homemade
from CNN_layers import linear_layer_homemade
from CNN_layers import elu_homemade
from CNN_layers import relu_homemade as ReLU
from Tests.helper_functions import tokenize, transform_input, compare, create_conv_homemade, \
                                create_batchnorm_homemade, create_maxpool_homemade

In [34]:
MNIST_test = datasets.MNIST(root='./data', train=True, download=True, transform=torchvision.transforms.ToTensor())
test_set = [[data[0], tokenize(data[1])] for data in MNIST_test if data[1] in [1,2]]

batch_size = 1000
test_loader = DataLoader(test_set, batch_size=batch_size)

In [35]:
model_original = CNNSmall()
model_original.eval()

path = "CNN_small"
load = True

if load and os.path.isfile(path):
    model_original.load_state_dict(torch.load(path))

In [36]:
input_original_test, label_test = next(iter(test_loader))
input_homemade_test = transform_input(input_batch=input_original_test)

In [37]:
elu_homemade = elu_homemade.ELU()
relu_homemade = ReLU.relu()
conv1_homemade = create_conv_homemade(model_conv= model_original.conv1)
batchnorm1_homemade = create_batchnorm_homemade(model_batchnorm= model_original.batchNorm1)
maxpool1_homemade = create_maxpool_homemade(model_maxpool= model_original.maxPool1)
conv2_homemade = create_conv_homemade(model_conv= model_original.conv2)
batchnorm2_homemade = create_batchnorm_homemade(model_batchnorm= model_original.batchNorm2)
maxpool2_homemade = create_maxpool_homemade(model_original.maxPool2)
linear_homemade = linear_layer_homemade.linear_layer(model_original.lin.weight,model_original.lin.bias,2)

STRIDE ER PRÆDEFINERET
STRIDE ER PRÆDEFINERET


## Shape of the network architecture

In [38]:
print(model_original)

CNNSmall(
  (conv1): Conv2d(1, 3, kernel_size=(3, 3), stride=(1, 1))
  (batchNorm1): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu1): ReLU()
  (maxPool1): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(3, 5, kernel_size=(5, 5), stride=(1, 1))
  (batchNorm2): BatchNorm2d(5, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu2): ReLU()
  (maxPool2): MaxPool2d(kernel_size=(3, 3), stride=(3, 3), padding=0, dilation=1, ceil_mode=False)
  (lin): Linear(in_features=45, out_features=2, bias=True)
  (network): Sequential(
    (0): Conv2d(1, 3, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(3, 5, kernel_size=(5, 5), stride=(1, 1))
    (5): BatchNorm2d(5, eps=1e-05, momentum=0.1, aff

## Testing the first convolutional layer

In [39]:
# homemade conv1 filter on test
out_homemade = conv1_homemade(input_homemade_test)
# original conv1 filter on test
out_original = model_original.conv1(input_original_test)

print("Convolutional layer 1 error over out channels: ")
compare(out_homemade, transform_input(input_batch = out_original))

Convolutional layer 1 error over out channels: 


([[2.714658446367757e-06, 2.784962947907643e-06, 1.9446338829220977e-06],
  [5.921787107621146e-06, 4.750881298247467e-06, 3.3989394729316835e-06],
  [3.410857115693222e-06, 2.2393274963089516e-06, 1.834238120057563e-06],
  [1.971447044196717e-06, 1.861740862883765e-06, 1.6844635346846548e-06],
  [2.1549792094399667e-06, 1.6317833798004666e-06, 1.600185750155747e-06],
  [4.3209848331615675e-06, 3.980241721310285e-06, 3.24398926659919e-06],
  [2.7404191907942632e-06, 2.8128384644901527e-06, 1.9174731725468686e-06],
  [3.5680181167396796e-06, 4.050671866123734e-06, 3.024474747675976e-06],
  [6.341285260558216e-06, 6.487700503342297e-06, 4.056592588175256e-06],
  [5.925121876554362e-06, 7.271772738896842e-06, 3.804604128160305e-06],
  [2.0863131358350717e-06, 2.5371145600319722e-06, 1.6611967727231391e-06],
  [3.0308394650679915e-06, 2.811302073406352e-06, 1.9890298071922885e-06],
  [3.1337774651363226e-06, 2.2272761876565572e-06, 2.264267921103791e-06],
  [3.893594192727612e-06, 3.621131

## Testing the first Batchnorm layer

In [40]:
# homemade batchNorm1 filter on test
out_homemade = batchnorm1_homemade(input_batch=out_homemade)
# original batchNorm1 filter on test
out_original = model_original.batchNorm1(out_original)

print("BatchNorm layer 1 error over out channels: ")
compare(out_homemade, transform_input(input_batch = out_original))

BatchNorm layer 1 error over out channels: 


([[4.330277442932129e-05, 2.0656734704971313e-05, 7.69440084695816e-05],
  [5.023926496505737e-05, 3.732740879058838e-05, 7.526762783527374e-05],
  [4.6446919441223145e-05, 1.488998532295227e-05, 8.211284875869751e-05],
  [4.311650991439819e-05, 1.2528151273727417e-05, 7.922947406768799e-05],
  [4.176050424575806e-05, 1.2088567018508911e-05, 7.910281419754028e-05],
  [4.630535840988159e-05, 3.852136433124542e-05, 7.543712854385376e-05],
  [4.491955041885376e-05, 2.189353108406067e-05, 7.567740976810455e-05],
  [4.369020462036133e-05, 3.1303614377975464e-05, 7.627718150615692e-05],
  [4.89652156829834e-05, 4.8959627747535706e-05, 7.348135113716125e-05],
  [4.906952381134033e-05, 5.25936484336853e-05, 7.458403706550598e-05],
  [4.2691826820373535e-05, 1.656077802181244e-05, 7.797777652740479e-05],
  [4.4696033000946045e-05, 2.1532177925109863e-05, 7.937848567962646e-05],
  [4.540383815765381e-05, 1.864321529865265e-05, 7.335841655731201e-05],
  [4.7169625759124756e-05, 2.779066562652588e

## Testing the first ReLU layer

In [41]:
out_homemade = relu_homemade(out_homemade)

out_original = model_original.relu1(out_original)

print("ReLU layer 1 error over out channels")
compare(out_homemade, transform_input(input_batch = out_original))

ReLU layer 1 error over out channels


([[8.52346420288086e-06, 1.2625008821487427e-05, 6.531365215778351e-05],
  [1.8537044525146484e-05, 1.92299485206604e-05, 5.605258047580719e-05],
  [1.1324882507324219e-05, 9.8496675491333e-06, 7.034093141555786e-05],
  [6.094574928283691e-06, 6.496906280517578e-06, 7.144734263420105e-05],
  [4.880130290985107e-06, 5.844980478286743e-06, 7.17751681804657e-05],
  [1.3805925846099854e-05, 1.819618046283722e-05, 5.804374814033508e-05],
  [8.672475814819336e-06, 1.401454210281372e-05, 6.514228880405426e-05],
  [1.0229647159576416e-05, 1.5683472156524658e-05, 6.281770765781403e-05],
  [2.1509826183319092e-05, 2.7274712920188904e-05, 4.950910806655884e-05],
  [2.1204352378845215e-05, 3.249943256378174e-05, 5.1178038120269775e-05],
  [5.900859832763672e-06, 1.1177733540534973e-05, 6.887689232826233e-05],
  [9.395182132720947e-06, 1.3444572687149048e-05, 6.514415144920349e-05],
  [1.0870397090911865e-05, 1.097656786441803e-05, 6.51925802230835e-05],
  [1.2122094631195068e-05, 1.623481512069702

## Testing the first Maxpool layer

In [42]:
# homemade MaxPool filter on test
out_homemade = maxpool1_homemade(input_batch=out_homemade)
# original MaxPool filter on test
out_original = model_original.maxPool1(out_original)

print("MaxPool layer 1 error over out channels: ")
compare(out_homemade, transform_input(input_batch = out_original))

MaxPool layer 1 error over out channels: 


([[3.3602118492126465e-06, 6.295740604400635e-06, 1.7449259757995605e-05],
  [7.860362529754639e-06, 8.106231689453125e-06, 1.5363097190856934e-05],
  [4.813075065612793e-06, 2.4549663066864014e-06, 1.893937587738037e-05],
  [1.9818544387817383e-06, 2.205371856689453e-06, 1.8388032913208008e-05],
  [1.9669532775878906e-06, 2.5369226932525635e-06, 1.8905848264694214e-05],
  [4.61190938949585e-06, 7.787719368934631e-06, 1.516193151473999e-05],
  [3.255903720855713e-06, 5.494803190231323e-06, 1.755356788635254e-05],
  [4.336237907409668e-06, 7.010996341705322e-06, 1.7039477825164795e-05],
  [7.3015689849853516e-06, 1.027621328830719e-05, 1.4450401067733765e-05],
  [6.549060344696045e-06, 1.1516734957695007e-05, 1.436471939086914e-05],
  [2.6226043701171875e-06, 4.364177584648132e-06, 1.7687678337097168e-05],
  [3.2410025596618652e-06, 5.681067705154419e-06, 1.6987323760986328e-05],
  [3.926455974578857e-06, 4.410743713378906e-06, 1.786835491657257e-05],
  [4.597008228302002e-06, 6.1951577

## Testing the second Convolutional layer

In [43]:
# TODO: test the second convolutional layer

# homemade batchNorm1 filter on test
out_homemade = conv2_homemade(input_batch=out_homemade)
# original batchNorm1 filter on test
out_original = model_original.conv2(out_original)

print("MaxPool layer 1 error over out channels: ")
compare(out_homemade, transform_input(input_batch = out_original))

MaxPool layer 1 error over out channels: 


([[7.265901361658278e-06,
   6.0008974384351266e-06,
   6.8729584533633314e-06,
   8.080163430713994e-06,
   6.1661622124398185e-06],
  [9.889932549300706e-06,
   8.173251276083482e-06,
   8.50474784619415e-06,
   1.0051189772125602e-05,
   7.640689441951176e-06],
  [4.669783203833933e-06,
   6.234225281120942e-06,
   4.89284816798663e-06,
   6.749783225679096e-06,
   4.869403133156425e-06],
  [4.318607942380148e-06,
   4.908561359595609e-06,
   4.291669769007633e-06,
   7.06738558492459e-06,
   3.3300237545497158e-06],
  [4.265040094070249e-06,
   4.99849311685896e-06,
   4.759057099429365e-06,
   7.843769908061349e-06,
   3.8455810851129035e-06],
  [7.534773463935274e-06,
   6.374395943961608e-06,
   7.048334270093326e-06,
   9.25581891011995e-06,
   8.98312575860205e-06],
  [5.797371207955093e-06,
   5.491700898294094e-06,
   5.127616301392468e-06,
   7.821658449196078e-06,
   6.493704068660788e-06],
  [8.489942708411746e-06,
   8.041826404583952e-06,
   8.212692064246685e-06,
   9.

## Testing the second Batchnorm layer

In [44]:
# homemade batchNorm1 filter on test
out_homemade = batchnorm2_homemade(input_batch=out_homemade)
# original batchNorm1 filter on test
out_original = model_original.batchNorm2(out_original)

print("MaxPool layer 1 error over out channels: ")
compare(out_homemade, transform_input(input_batch = out_original))

MaxPool layer 1 error over out channels: 


([[2.3447908461093903e-05,
   2.220645546913147e-05,
   2.6049092411994934e-05,
   2.2366642951965332e-05,
   1.528114080429077e-05],
  [3.1970441341400146e-05,
   3.1556934118270874e-05,
   2.845749258995056e-05,
   2.7514994144439697e-05,
   1.93864107131958e-05],
  [1.5756115317344666e-05,
   2.3499131202697754e-05,
   2.1282583475112915e-05,
   1.8388032913208008e-05,
   1.111254096031189e-05],
  [1.6026198863983154e-05,
   1.9669532775878906e-05,
   2.1537765860557556e-05,
   1.9729137420654297e-05,
   8.009374141693115e-06],
  [1.5147030353546143e-05,
   1.749396324157715e-05,
   2.3268163204193115e-05,
   2.2530555725097656e-05,
   9.395182132720947e-06],
  [2.3789703845977783e-05,
   2.5033950805664062e-05,
   2.4529173970222473e-05,
   2.6457011699676514e-05,
   2.3826956748962402e-05],
  [1.8211081624031067e-05,
   2.1476298570632935e-05,
   1.935567706823349e-05,
   2.1383166313171387e-05,
   1.7739832401275635e-05],
  [2.6151537895202637e-05,
   2.946704626083374e-05,
   2.

## Testing the second ReLU

In [45]:
out_homemade = relu_homemade(out_homemade)

out_original = model_original.relu1(out_original)

print("ReLU layer 1 error over out channels")
compare(out_homemade, transform_input(input_batch = out_original))

ReLU layer 1 error over out channels


([[6.916932761669159e-06,
   1.3154000043869019e-05,
   4.015862941741943e-06,
   9.581446647644043e-06,
   8.456408977508545e-06],
  [2.539902925491333e-05,
   1.570954918861389e-05,
   2.096220850944519e-05,
   8.441507816314697e-06,
   1.1064112186431885e-05],
  [1.9073486328125e-06,
   1.039355993270874e-05,
   5.401670932769775e-07,
   1.0132789611816406e-05,
   7.99819827079773e-06],
  [1.3373792171478271e-06,
   8.866190910339355e-06,
   1.391395926475525e-06,
   1.4632940292358398e-05,
   5.207955837249756e-06],
  [7.152557373046875e-07,
   4.395842552185059e-06,
   3.203749656677246e-07,
   1.4275312423706055e-05,
   6.861984729766846e-06],
  [1.6555190086364746e-05,
   1.0967254638671875e-05,
   1.5361234545707703e-05,
   1.0214745998382568e-05,
   1.516193151473999e-05],
  [3.8035213947296143e-06,
   1.0844320058822632e-05,
   3.5688281059265137e-06,
   8.508563041687012e-06,
   1.208484172821045e-05],
  [1.9922852516174316e-05,
   1.3872981071472168e-05,
   1.90623104572296

## Testing the second Maxpool layer

In [46]:
# homemade batchNorm1 filter on test
out_homemade = maxpool2_homemade(input_batch=out_homemade)
# original batchNorm1 filter on test
out_original = model_original.maxPool2(out_original)

print("MaxPool layer 1 error over out channels: ")
compare(out_homemade, transform_input(input_batch = out_original))

MaxPool layer 1 error over out channels: 


([[2.7641654014587402e-06,
   2.9802322387695312e-06,
   1.996755599975586e-06,
   1.9669532775878906e-06,
   2.3543834686279297e-06],
  [4.76837158203125e-06,
   3.129243850708008e-06,
   3.933906555175781e-06,
   1.1324882507324219e-06,
   2.816319465637207e-06],
  [1.0132789611816406e-06,
   3.1813979148864746e-06,
   3.5762786865234375e-07,
   1.3113021850585938e-06,
   1.642853021621704e-06],
  [4.917383193969727e-07,
   3.4570693969726562e-06,
   7.655471563339233e-07,
   1.9073486328125e-06,
   6.705522537231445e-07],
  [2.980232238769531e-07,
   9.834766387939453e-07,
   3.203749656677246e-07,
   1.7881393432617188e-06,
   1.6093254089355469e-06],
  [3.635883331298828e-06,
   2.771615982055664e-06,
   2.6226043701171875e-06,
   2.682209014892578e-06,
   6.258487701416016e-06],
  [8.344650268554688e-07,
   1.7285346984863281e-06,
   1.6242265701293945e-06,
   1.4603137969970703e-06,
   3.0547380447387695e-06],
  [5.9604644775390625e-06,
   2.682209014892578e-06,
   4.79817390441

## Testing the Linear layer

In [47]:
# homemade linear filter on test
out_homemade = linear_homemade(torch.reshape(torch.tensor(out_homemade, dtype = torch.double),(batch_size,45)))
# original linear filter on test
out_original = model_original.lin(torch.reshape(out_original, (batch_size,45)))

print("Linear layer 1 error over out channels: ")
compare(out_homemade.numpy(), list(list(out_original.detach().numpy())))

Linear layer 1 error over out channels: 


([[1.851302506850061e-07, 2.729733328266093e-07],
  [6.596299417793716e-07, 2.2939073041072788e-07],
  [1.536124525358673e-07, 1.0843013900085907e-06],
  [1.908156299634811e-08, 4.607633554698509e-07],
  [3.2989721177045794e-07, 2.2511085457921354e-06],
  [3.279055569294087e-07, 7.775501558171527e-07],
  [7.553327678877508e-07, 1.9772788526495333e-06],
  [2.265407017532084e-08, 6.694520715555541e-07],
  [1.8432992234806989e-06, 1.945864653407625e-06],
  [2.8769920312754493e-07, 5.419140425289015e-07],
  [8.749102295979583e-07, 1.4055674757429415e-06],
  [9.709592552198387e-07, 1.6135263525285382e-07],
  [1.94342754689103e-06, 1.5045452403938953e-06],
  [6.060612900427031e-08, 4.83358034486514e-07],
  [1.930284323492515e-07, 9.50704421853743e-08],
  [1.539042756348863e-06, 9.81713611736268e-07],
  [1.116784046573116e-06, 1.123192689078678e-06],
  [7.5816642475246e-07, 8.214897739833305e-08],
  [2.8828948188674985e-06, 7.085761009051339e-07],
  [5.197209214102827e-07, 5.422441828883962e-

In [54]:
for i in range(batch_size):
    assert(np.argmax(out_homemade[i]) == np.argmax(out_original[i].detach().numpy()))